# Web Scraping - Lab

## Introduction

Now that you've seen a more extensive example of developing a web scraping script, it's time to further practice and formalize that knowledge by writing functions to parse specific pieces of information from the web page and then synthesizing these into a larger loop that will iterate over successive web pages in order to build a complete dataset.

## Objectives

You will be able to:

* Write functions to parse specific information from a web page
* Iterate over successive web pages in order to create a dataset

## Lab Overview

This lab will build upon the previous lesson. In the end, you'll look to write a script that will iterate over all of the pages for the demo site and extract the title, price, star rating and availability of each book listed. Building up to that, you'll formalize the concepts from the lesson by writing functions that will extract a list of each of these features for each web page. You'll then combine these functions into the full script which will look something like this:  

```python
df = pd.DataFrame()
for i in range(2,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_star_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    ...
 ```

In [68]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from urllib.request import urlopen
from word2number import w2n

In [10]:
url = "http://books.toscrape.com/catalogue/page-1.html"
books = requests.get(url)

In [11]:
soup = BeautifulSoup(books.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:30" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="../static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="../static/oscar/css/styles.css" rel="stylesheet" typ

## Retrieving Titles

To start, write a function that extracts the titles of the books on a given page. The input for the function should be the `soup` for the HTML of the page.

In [16]:
def retrieve_titles(soup):
    titles = []
    for book in soup.find_all("a", attrs={"title":True}):
        titles.append(book.get("title"))
    return titles

## Retrieve Ratings

Next, write a similar function to retrieve the star ratings on a given page. Again, the function should take in the `soup` from the given html page and return a list of the star-ratings for the books. These star ratings should be formatted as integers.

In [41]:
def retrieve_ratings(soup):
    ratings = []
    for book in soup.find_all("p", attrs={"class":"star-rating"}):
        book = str(book).split("\"")[1].split(" ")[1]
        rating = w2n.word_to_num(book)
        ratings.append(rating)
    return ratings

## Retrieve Prices

Now write a function to retrieve the prices on a given page. The function should take in the `soup` from the given page and return a list of prices formatted as floats.

In [45]:
def retrieve_prices(soup):
    prices = []
    for book in soup.find_all("p", attrs={"class":"price_color"}):
        book = float(book.text[1:])
        prices.append(book)
    return prices

## Retrieve Availability

Write a function to retrieve whether each book is available or not. The function should take in the `soup` from a given html page and return a list of the availability for each book.

In [65]:
def retrieve_avails(soup):
    avails = []
    for book in soup.find_all("p", attrs={"class":"instock availability"}):
        book = " ".join(str(book).split(">")[3].strip().split(" ")[:2]).rstrip()
        avails.append(book)
    return avails

## Create a Script to Retrieve All the Books From All 50 Pages

Finally, write a script to retrieve all of the information from all 50 pages of the books.toscrape.com website. 

In [80]:
df = pd.DataFrame()
for i in range(1,51):
    url = "http://books.toscrape.com/catalogue/page-{}.html".format(i)
    new_page = requests.get(url)
    soup = BeautifulSoup(new_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    new_df = pd.DataFrame([new_titles, new_ratings, new_prices, new_avails]).T
    df = pd.concat([df, new_df])
df.columns = ["Book Title", "Rating", "Price", "Availability"]
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)

In [86]:
df.head()

,Book Title,Rating,Price,Availability
0,A Light in the Attic,3,51.77,In stock
1,Tipping the Velvet,1,53.74,In stock
2,Soumission,1,50.1,In stock
3,Sharp Objects,4,47.82,In stock
4,Sapiens: A Brief History of Humankind,5,54.23,In stock


## Level-Up: Write a new version of the script you just wrote. 

If you used url hacking to generate each successive page url, instead write a function that retrieves the link from the `"next"` button at the bottom of the page. Conversely, if you already used this approach above, use URL-hacking (arguably the easier of the two methods in this case).

In [88]:
url = "http://books.toscrape.com/catalogue/page-1.html"
books = requests.get(url)
soup = BeautifulSoup(books.content, 'html.parser')

In [94]:
def retrieve_link(soup):
    link_end = soup.find_all("li", attrs={"class":"next"})[0]
    link_end = str(link_end).split("\"")[3]
    next_page = "http://books.toscrape.com/catalogue/" + link_end
    return next_page

In [95]:
retrieve_link(soup)

'http://books.toscrape.com/catalogue/page-2.html'

In [106]:
df = pd.DataFrame()
url = "http://books.toscrape.com/catalogue/page-1.html"
while len(df) < 1000:
    new_page = requests.get(url)
    soup = BeautifulSoup(new_page.content, 'html.parser')
    new_titles = retrieve_titles(soup)
    new_ratings = retrieve_ratings(soup)
    new_prices = retrieve_prices(soup)
    new_avails = retrieve_avails(soup)
    new_df = pd.DataFrame([new_titles, new_ratings, new_prices, new_avails]).T
    df = pd.concat([df, new_df])
    try:
        url = retrieve_link(soup)
    except:
        pass
df.columns = ["Book Title", "Rating", "Price", "Availability"]
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)

In [107]:
df.tail()

,Book Title,Rating,Price,Availability
995,Alice in Wonderland (Alice's Adventures in Won...,1,55.53,In stock
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",4,57.06,In stock
997,A Spy's Devotion (The Regency Spies of London #1),5,16.97,In stock
998,1st to Die (Women's Murder Club #1),1,53.98,In stock
999,"1,000 Places to See Before You Die",5,26.08,In stock


## Summary

Well done! You just completed your first full web scraping project! You're ready to start harnessing the power of the web!